In [2]:
#Декоратор trace выводит на экран сообщение с информацией о вызове декорируемой функции.
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner
#Применим его к тождественной функции
@trace
def identity(x):
    "I do nothing useful."
    return x
identity(42)

identity (42,) {}


42

In [1]:
#чтоб (__name__, __doc__) функции стали доступны после работы декоратора используем @functools.wraps(func)
import functools
def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

In [3]:
#Заведём глобальную переменную trace_enabled и с её помощью будем отключать и включать trace.
#Если trace выключен, то результатом применения декоратора является сама функция func — никаких
#дополнительных действий в момент её исполнения производиться не будет
trace_enabled = False
def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner if trace_enabled else func

In [8]:
#Декораторы с аргументами: реализация
def trace(handle):
    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs,'file=',handle)
            return func(*args, **kwargs)
        return inner
    return decorator
@trace(12)
def foo(x):
    return x
foo(100)

foo (100,) {} file= 12


100

In [12]:
#Декораторы с аргументами: человечный trace
#Что происходит:
#1. with_arguments принимает декоратор deco,
#2. заворачивает его во wrapper, так как deco — декоратор с аргументами, а затем в decorator.
#3. decorator конструирует новый декоратор с помощью deco и копирует в него внутренние атрибуты функции func.
def with_arguments(deco):
    @functools.wraps(deco)
    def wrapper(*dargs, **dkwargs):
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            functools.update_wrapper(result, func)
            return result
        return decorator
    return wrapper

@with_arguments
def trace(func, handle):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs, 'file=',handle)
        return func(*args, **kwargs)
    return inner

@trace(12)
def identity(x):
    return x
identity(100)

identity (100,) {} file= 12


100

In [14]:
#Декораторы с опциональными аргументами
def trace(func=None, *, handle=12):
    # со скобками
    if func is None:
        return lambda func: trace(func, handle=handle)
    # без скобок
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

# EXAMPLES

In [23]:
#засечь время работы на каком-то числе прогонок
import time
def timethis(func=None, *, n_iter=100):
    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)

    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")
        acc = float("inf")
        for i in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)
        return result
    return inner

@timethis
def ss(n):
    return sum(n)

ss(range(10**4))

ss ... 0.0004612764414275716


49995000

In [24]:
#профилирование - сколько раз была вызвана ф-ция
def profiled(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    inner.ncalls = 0
    return inner

@profiled
def identity(x):
    return x

print(identity(42))
print(identity.ncalls)

42
1


In [27]:
#Контрактное программирование — способ проектирования программ, основывающийся на формальном описании
#интерфейсов в терминах предусловий, постусловий и инвариантов.
#В Python контрактное программирование можно реализовать в виде библиотеки декораторов

def pre(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            assert cond(*args, **kwargs), message
            return func(*args, **kwargs)
        return inner
    return wrapper

@pre(lambda x: x >= 0, "negative argument")
def checked_log(x):
    return math.log(x)
checked_log(-42)

AssertionError: negative argument

In [29]:
#post
import math
def post(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            result = func(*args, **kwargs)
            assert cond(result), message
            return result
        return inner
    return wrapper

@post(lambda x: not math.isnan(x), "not a number")
def something_useful():
    return float("nan")

something_useful()

AssertionError: not a number

# Модуль functools

In [30]:
#Родственник уже рассмотренного memoized, сохраняющий фиксированное количество последних вызовов
@functools.lru_cache(maxsize=64)
def ackermann(m, n):
    pass
ackermann.cache_info() #cache информация - оч полезно

CacheInfo(hits=0, misses=0, maxsize=64, currsize=0)

In [31]:
#Функция reduce обобщает логику функции sum на произвольную бинарную операцию.
functools.reduce(lambda acc, x: acc * x,[1, 2, 3, 4])

24